# Google Research Football

Exploration of *Rule Based and Reinforcement Learning Hybrid solution* for the Kaggle Competition: **Google Research Football with Manchester City F.C.**

The approach is as follows, decompose the agent into specific cases (like a freekick or a 1v1) and use RL agents trained on custom scenarios for optimal decison making.

# Installs

In [ ]:
%%bash
# Kaggle environments.
git clone https://github.com/Kaggle/kaggle-environments.git
cd kaggle-environments && pip install .

# GFootball environment.
apt-get update -y
apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
git clone -b v2.7 https://github.com/google-research/football.git
mkdir -p football/third_party/gfootball_engine/lib
wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.7.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so

# Custom Scenario

The game already has pre-built scenarios ([Repository](https://github.com/google-research/football/tree/master/gfootball/scenarios)). The Environment enables us to build custom scenarios. A scenario is a football game with specific characteristics such a limited number of players. For the scenario to be included in the environment, we must write into a file as such and build the game afterwards. The following gist shows how to create a custom scenario. 

**Select the options to customize (not all are needed) and change the variable (in capital letters), including the file name, to be the desired parameters.**

```python
%%writefile kaggle-environments/football/gfootball/scenarios/CUSTOM_NAME.py
# Imports 
from . import *

def build_scenario(builder):
    # Game Duration
    builder.config().game_duration = INTEGER
    builder.config().second_half = INTEGER
    
    # Default team decison making
    builder.config().right_team_difficulty = FLOAT #(Between 0 and 1)
    builder.config().left_team_difficulty = 1.0
    
    # Does the game have stochastic event 
    builder.config().deterministic = BOOLEAN
    
    # Determing which side to be on (Optional)
    # If using this, set the teams accordingly: builder.SetTeam(first_team)
    if builder.EpisodeNumber() % 2 == 0:
        first_team = Team.e_Left
        second_team = Team.e_Right
    else:
        first_team = Team.e_Right
        second_team = Team.e_Left
    
    # Consider Offside Rule
    builder.config().offsides = BOOLEAN # Default = True
    
    # Conditions to end episodes
    builder.config().end_episode_on_score = BOOLEAN # Default = False
    builder.config().end_episode_on_out_of_play = BOOLEAN # Default = False
    builder.config().end_episode_on_possession_change = BOOLEAN # Default = False
    
    # Custom ball position
    builder.SetBallPosition(XPOS, YPOS)
    
    # Create Left Team
    builder.SetTeam(Team.e_Left)
    builder.AddPlayer(X, Y, ROLE, LAZY, CONTROLLABLE) 
    # x: x coordinate of the player in the range [-1, 1].
    # y: y coordinate of the player in the range [-0.42, 0.42].
    # role: Player's role in the game (goal keeper etc.).
    # lazy (Optional BOOLEAN default False): Computer doesn't perform any automatic actions for lazy player.
    # controllable (Optional BOOLEAN default True): Whether player can be controlled.

    # Create Right Team
    builder.SetTeam(Team.e_Right)
    builder.AddPlayer(-1.0, 0.0, e_PlayerRole_GK,)
    builder.AddPlayer(-0.7, 0.0, e_PlayerRole_CB)
```

In [ ]:
# This variable can be used for fancy animation.
# Make sure its value is the same as (or at least smaller than) the one in the scenario. 
game_duration = 3000

Let us build a Scenario identical as that of the competition but make it a deterministic environment (better for debugging).


In [ ]:
%%writefile kaggle-environments/football/gfootball/scenarios/custom_scenario_11v11.py
from . import *

def build_scenario(builder):
    builder.config().game_duration = 3000
    builder.config().right_team_difficulty = 1.0
    builder.config().left_team_difficulty = 1.0
    builder.config().deterministic = True
    if builder.EpisodeNumber() % 2 == 0:
        first_team = Team.e_Left
        second_team = Team.e_Right
    else:
        first_team = Team.e_Right
        second_team = Team.e_Left
    builder.SetTeam(first_team)
    builder.AddPlayer(-1.000000, 0.000000, e_PlayerRole_GK)
    builder.AddPlayer(0.000000,  0.020000, e_PlayerRole_RM)
    builder.AddPlayer(0.000000, -0.020000, e_PlayerRole_CF)
    builder.AddPlayer(-0.422000, -0.19576, e_PlayerRole_LB)
    builder.AddPlayer(-0.500000, -0.06356, e_PlayerRole_CB)
    builder.AddPlayer(-0.500000, 0.063559, e_PlayerRole_CB)
    builder.AddPlayer(-0.422000, 0.195760, e_PlayerRole_RB)
    builder.AddPlayer(-0.184212, -0.10568, e_PlayerRole_CM)
    builder.AddPlayer(-0.267574, 0.000000, e_PlayerRole_CM)
    builder.AddPlayer(-0.184212, 0.105680, e_PlayerRole_CM)
    builder.AddPlayer(-0.010000, -0.21610, e_PlayerRole_LM)
    builder.SetTeam(second_team)
    builder.AddPlayer(-1.000000, 0.000000, e_PlayerRole_GK)
    builder.AddPlayer(-0.050000, 0.000000, e_PlayerRole_RM)
    builder.AddPlayer(-0.010000, 0.216102, e_PlayerRole_CF)
    builder.AddPlayer(-0.422000, -0.19576, e_PlayerRole_LB)
    builder.AddPlayer(-0.500000, -0.06356, e_PlayerRole_CB)
    builder.AddPlayer(-0.500000, 0.063559, e_PlayerRole_CB)
    builder.AddPlayer(-0.422000, 0.195760, e_PlayerRole_RB)
    builder.AddPlayer(-0.184212, -0.10568, e_PlayerRole_CM)
    builder.AddPlayer(-0.267574, 0.000000, e_PlayerRole_CM)
    builder.AddPlayer(-0.184212, 0.105680, e_PlayerRole_CM)
    builder.AddPlayer(-0.010000, -0.21610, e_PlayerRole_LM)

Now we can build the environment with the custom scenarios built into it.

In [ ]:
%%bash
# Install
cd kaggle-environments/football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

In [ ]:
%%bash
# Proof of Concept: verify the presence of the desired files
find / -name custom_scenario*.py | grep site-packages | cut -c -90

# Evaluation

[Started Bot](https://www.kaggle.com/c/google-football/overview/getting-started)

In [ ]:
%%writefile evaluate.py

from kaggle_environments.envs.football.helpers import *

# @human_readable_agent wrapper modifies raw observations 
# provided by the environment:
# https://github.com/google-research/football/blob/master/gfootball/doc/observation.md#raw-observations
# into a form easier to work with by humans.
# Following modifications are applied:
# - Action, PlayerRole and GameMode enums are introduced.
# - 'sticky_actions' are turned into a set of active actions (Action enum)
#    see usage example below.
# - 'game_mode' is turned into GameMode enum.
# - 'designated' field is removed, as it always equals to 'active'
#    when a single player is controlled on the team.
# - 'left_team_roles'/'right_team_roles' are turned into PlayerRole enums.
# - Action enum is to be returned by the agent function.
@human_readable_agent
def agent(obs):
    # Make sure player is running.
    if Action.Sprint not in obs['sticky_actions']:
        return Action.Sprint
    # We always control left team (observations and actions
    # are mirrored appropriately by the environment).
    controlled_player_pos = obs['left_team'][obs['active']]
    # Does the player we control have the ball?
    if obs['ball_owned_player'] == obs['active'] and obs['ball_owned_team'] == 0:
        # Shot if we are 'close' to the goal (based on 'x' coordinate).
        if controlled_player_pos[0] > 0.5:
            return Action.Shot
        # Run towards the goal otherwise.
        return Action.Right
    else:
        # Run towards the ball.
        if obs['ball'][0] > controlled_player_pos[0] + 0.05:
            return Action.Right
        if obs['ball'][0] < controlled_player_pos[0] - 0.05:
            return Action.Left
        if obs['ball'][1] > controlled_player_pos[1] + 0.05:
            return Action.Bottom
        if obs['ball'][1] < controlled_player_pos[1] - 0.05:
            return Action.Top
        # Try to take over the ball if close to the ball.
        return Action.Slide

[Tunable Baseline Bot](https://www.kaggle.com/david1013/tunable-baseline-bot) bot to evaluate our agent.

In [ ]:
%%writefile evaluate.py

# Tune Here:
SPRINT_RANGE = 0.6

SHOT_RANGE_X = 0.7  
SHOT_RANGE_Y = 0.2

GOALIE_OUT = 0.2
LONG_SHOT_X = 0.4
LONG_SHOT_Y = 0.2

from kaggle_environments.envs.football.helpers import *
from math import sqrt

directions = [
[Action.TopLeft, Action.Top, Action.TopRight],
[Action.Left, Action.Idle, Action.Right],
[Action.BottomLeft, Action.Bottom, Action.BottomRight]]

dirsign = lambda x: 1 if abs(x) < 0.01 else (0 if x < 0 else 2)

enemyGoal = [1, 0]
GOALKEEPER = 0

shot_range = [[SHOT_RANGE_X, 1], 
                  [-SHOT_RANGE_Y, SHOT_RANGE_Y]]


def inside(pos, area):
    return area[0][0] <= pos[0] <= area[0][1] and area[1][0] <= pos[1] <= area[1][1]

@human_readable_agent
def agent(obs):
    controlled_player_pos = obs['left_team'][obs['active']]
    
    if obs["game_mode"] == GameMode.Penalty:
        return Action.Shot
    if obs["game_mode"] == GameMode.Corner:
        if controlled_player_pos[0] > 0:
            return Action.Shot
    if obs["game_mode"] == GameMode.FreeKick:
        return Action.Shot
    
    # Make sure player is running down the field.
    if  0 < controlled_player_pos[0] < SPRINT_RANGE and Action.Sprint not in obs['sticky_actions']:
        return Action.Sprint
    elif SPRINT_RANGE < controlled_player_pos[0] and Action.Sprint in obs['sticky_actions']:
        return Action.ReleaseSprint

    # If our player controls the ball:
    if obs['ball_owned_player'] == obs['active'] and obs['ball_owned_team'] == 0:
        
        if inside(controlled_player_pos, shot_range) and controlled_player_pos[0] < obs['ball'][0]:
            return Action.Shot
        
        elif ( abs(obs['right_team'][GOALKEEPER][0] - 1) > GOALIE_OUT   
                and controlled_player_pos[0] > LONG_SHOT_X and abs(controlled_player_pos[1]) < LONG_SHOT_Y ):
            return Action.Shot
        
        else:
            xdir = dirsign(enemyGoal[0] - controlled_player_pos[0])
            ydir = dirsign(enemyGoal[1] - controlled_player_pos[1])
            return directions[ydir][xdir]
        
    # if we we do not have the ball:
    else:
        # Run towards the ball.
        xdir = dirsign(obs['ball'][0] - controlled_player_pos[0])
        ydir = dirsign(obs['ball'][1] - controlled_player_pos[1])
        return directions[ydir][xdir]

# Bot
[Source](https://www.kaggle.com/piotrstanczyk/gfootball-template-bot)

In [ ]:
%%writefile submission.py

from kaggle_environments.envs.football.helpers import *
import math
import numpy as np
from scipy.spatial.distance import cdist
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.geometry import LineString

@human_readable_agent
def agent(obs):

    def distance(coord_one, coord_two):
        ''' 
        Compute the distance between two players 
        Args:
            coord_one: Coordinates of the first entity (player, ball, goal ...)
            coord_two: Coordinates of the first entity
        Returns: Distance
        '''
        return np.linalg.norm(np.array(coord_one) - np.array(coord_two))

    def angle(coord_one, coord_two):
        ''' 
        Compute the angle between two coordinates
        Args:
            coord_one: Coordinates of the first entity (player, ball, goal ...)
            coord_two: Coordinates of the first entity
        Returns: Angle in degrees
        '''
        return np.degrees(np.arctan2(
            coord_two[1] - coord_one[1], coord_two[0] - coord_one[0]))
    
    def find_closest_opposition_to_ball():
        ''' 
        Find the opposing player closest to the ball
        Returns: Index of a opponent
        '''
        # Store distance to each opponent 
        distances = obs['right_team'].copy()
        for i, player_position in enumerate(obs['right_team']):
            # Compute the distance between the ball and the player
            distances[i] = distance(obs['ball'], player_position)
        # Get the index for which the distance is minimum
        return np.argmin(distances)
        
    def closest_player(player_position, team='left_team'):
        ''' 
        Find the player closest to the player in posession
        Args:
            player_position: Coordinates of the player in posession
            team: name of the team in which we look for a player. Use team = 'right_team' to get the closest opponent and use team = 'left_team' to get the closest teammate 
        Returns: Index of a player
        '''
        # Store distance to each player 
        distances = obs[team].copy()
        for i, teammate_position in enumerate(obs[team]):
            # Compute the distance between the ball and the player
            dist = distance(player_position, teammate_position)
            # Set infinite distance for the distance between the player controlled and himself
            distances[i] = np.Inf if dist == 0 else dist
        # Get the index for which the distance is minimum
        return np.argmin(distances)

    def coordinate_to_direction(coord_one, coord_two):
        ''' 
        Find cardinal direction to get from an entity to another
        Args:
            coord_one: Coordinates of the first entity (player, ball, goal ...)
            coord_two: Coordinates of the first entity
        Returns: Cardinal direction
        '''
        # Compute angle between entities
        deg = angle(coord_one, coord_two)
        # Convert to a direction
        if -22.5 <= deg and deg <= 22.5:
            return Action.Right
        elif 22.5 < deg and deg <= 67.5:
            return Action.BottomRight
        elif 67.5 < deg and deg <= 112.5:
            return Action.Bottom
        elif 112.5 <= deg and deg <= 157.5:
            return Action.BottomLeft
        elif deg > 157.5 or deg <= -157.5:
            return Action.Left
        elif deg <= -112.5:
            return Action.TopLeft
        elif deg <= -67.5:
            return Action.Top
        elif deg <= -22.5:
            return Action.TopRight

        return Action.Idle

    def direction_check(action, direction):
        ''' 
        Check that the player controlled is directed properly before we let it perform an action
        Args:
            action: One of 19 actions for the player
            direction: One of 9 actions
        Returns: Cardinal direction
        '''
        # Sticky is a N hot encoded array of size 19 for whether the player is performing an action
        # Like passing, shooting, runing right or left ...
        if direction in obs['sticky_actions']:
            # If the player is directed, perform an action
            return action
        else:
            return direction

    def expected_goal(player):
        ''' 
        Compute expected goals from paper :https://www.researchgate.net/publication/240641737_Estimating_the_probability_of_a_shot_resulting_in_a_goal_The_effects_of_distance_angle_and_space
        Args:
            player: Controlled player coordinates
        Returns: Float value representing probability of scoring
        '''
        # Football rules: size of goal : 8 yard by 8 feet
        to_yard = lambda coord: [coord[0] * 4 / 0.044, coord[1] * 4 / 0.044]
        # Get distance and angle to nearest post
        if (player[1] >= 0):
            # Case where closer to the top post
            yards_to_goal = distance(to_yard(player), to_yard([1, 0.044]))
            # Set the angle to 0 when in front of goal
            if (player[1] <= 0.044):
                angle_to_goal = 0
            else:
                angle_to_goal = abs(angle(player, [1, 0.044]))
        else:
            # Case where closer to the bot post
            yards_to_goal = distance(to_yard(player), to_yard([1, -0.044]))
            # Set the angle to 0 when in front of goal
            if (player[1] >= -0.044):
                angle_to_goal = 0
            else:
                angle_to_goal = abs(angle(player, [1, -0.044]))
        # Get distance to nearest opponent
        closest_opponent = closest_player(player, team='right_team')
        yards_to_closest_opponent = distance(player, obs['right_team'][closest_opponent]) * 4 / 0.044
        # Set boolean for whether the player has space to shoot
        space = 1 if yards_to_closest_opponent >= 1 else 0
        # Logistic regression equation
        y = 0.337 - 0.157 * yards_to_goal - 0.022 * angle_to_goal + 0.799 * space
        # Probability of scoring
        return np.exp(y) / (np.exp(y) + 1)
    
    def opponent_on_path(player, teammate):
        ''' 
        Find if there is an opponent player on the path of the path
        Args:
            player: Controlled player coordinates
            teammate: Teammate coordinates
        Returns: Boolean value for if there is a player obstructing
        '''
        body_radius = 0.012
        if player[0] > teammate[0] and player[1] > teammate[1]:
            # Case 3: BottomLeft pass
            polygon = Polygon([(player[0] + body_radius * 2, player[1] - body_radius * 2), 
                               (player[0] - body_radius * 2, player[1] + body_radius * 2),
                               (teammate[0] - body_radius * 2, teammate[1] + body_radius * 2),
                               (teammate[0] + body_radius * 2, teammate[1] - body_radius * 2)])
            
        elif player[0] < teammate[0] and player[1] > teammate[1]:
            # Case 2: BottomRight pass
            polygon = Polygon([(player[0] + body_radius * 2, player[1] + body_radius * 2), 
                               (player[0] - body_radius * 2, player[1] - body_radius * 2),
                               (teammate[0] - body_radius * 2, teammate[1] - body_radius * 2),
                               (teammate[0] + body_radius * 2, teammate[1] + body_radius * 2)])
            
        elif player[0] > teammate[0] and player[1] < teammate[1]:
            # Case 4: TopLeft pass
            polygon = Polygon([(player[0] + body_radius * 2, player[1] + body_radius * 2), 
                               (player[0] - body_radius * 2, player[1] - body_radius * 2),
                               (teammate[0] - body_radius * 2, teammate[1] - body_radius * 2),
                               (teammate[0] + body_radius * 2, teammate[1] + body_radius * 2)])
            
        elif player[0] < teammate[0] and player[1] < teammate[1]:
            # Case 1: TopRight pass
            polygon = Polygon([(player[0] - body_radius * 2, player[1] + body_radius * 2), 
                               (player[0] + body_radius * 2, player[1] - body_radius * 2),
                               (teammate[0] + body_radius * 2, teammate[1] - body_radius * 2),
                               (teammate[0] - body_radius * 2, teammate[1] + body_radius * 2)])
            
        else:
            # Same coordinate
            return False
            
        for opponent in enumerate(obs["right_team"]):
            point = Point(opponent[1][0], opponent[1][1])
            if polygon.contains(point):
                return True
        return False

    def move_to_position(pos, player_pos):
        '''
        Calculates vector between pos and player_pos, calculates cosine similarity to
        find the best direction action to get the player to the desired location in
        straight line fashion
        Args:
            pos: Desired destination position
            player_pos: Current player position
        Returns: Direction action
        '''
        if (len(pos) == 3):
            # 3D vector, convert to 2D
            pos = pos[:2]
        dir_array = dir_lookup["vectors"]
        desired_dir = (np.array(pos) - np.array(player_pos)).reshape(1, -1)

        # Map direction to closest action using cosine similarity
        cosine_dist = 1 - cdist(desired_dir, dir_array, metric='cosine')
        max_index = np.argmax(cosine_dist)
        return dir_lookup["actions"][max_index]

    def on_breakaway(player_pos, obs):
        '''Detects if a player is on a breakaway (aka 1v1 duel with goal) or not'''
        player_x = player_pos[0]
        right_team_pos = obs["right_team"][1:] # Not including the goalie
        return all([player_x > r_ply_pos[0] for r_ply_pos in right_team_pos])
    
    def projection(ball, player_pos):
        '''
        Compute the projection of the player coordinates onto the line formed
        by the ball position and the goal coordinates
        Args:
            ball: Current ball position
            player_pos: Current player position
        Returns: Direction action
        '''
        point = Point([player_pos[0], player_pos[1]])
        line = LineString([(-1, 0),(ball[0], ball[1])])

        x = np.array(point.coords[0])
        u = np.array(line.coords[0])
        v = np.array(line.coords[len(line.coords)-1])
        n = v - u
        n /= np.linalg.norm(n, 2)
        return u + n*np.dot(x-u, n)
    
    def predict_ball_pos(ball_pos, ball_direction, sprint = True):
        '''
        Predict next step's ball position given its direction and speed
        Args:
            ball_pos: Ball coordinates
            ball_direction: Ball movement in all directions
            sprint: Boolean for whether the player in posession is sprinting. 
            Cannot get opponent sticky so assume allways running.
        Returns: predicted ball position
        '''
        steps_run = 1/114
        steps_sprint = 1/77
        if sprint:
            return [ball_pos[:2][0] + (1+1/77) * ball_direction[:2][0], 
                    ball_pos[:2][1] + (1+1/77) * ball_direction[:2][1]]
        else:
            return [ball_pos[:2][0] + (1+1/114) * ball_direction[:2][0], 
                    ball_pos[:2][1] + (1+1/114) * ball_direction[:2][1]]
 
    ## Constants
    constants = {
        "delta_x": 9.2e-3,  # How much player / ball position changes when moving without sprinting
        "sprinting_delta_x": 1.33e-2,  # How much player / ball position changes when sprinting
        "steps_run": 1/114,
        "steps_sprint": 1/77,
        "max_distance_to_influence_ball": 0.01,
        "max_distance_to_influence_ball_sprinting": 0.0185,
        "pressure_threshold": 2/77,
        "goalie_out_of_box_dist": 0.2,
        "on_breakaway_goalie_dist": 0.2
    }

    state = {
        "prev_owner": None,
        "count": 10,
        "prev_player_pos": [0, 0],
        "prev_ball_pos": [0, 0, 0]
    }

    dir_lookup = {
        "vectors": np.array([[1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0], [-1, 1], [0, 1], [1, 1]]),
        "actions": [Action.Right, Action.TopRight, Action.Top, Action.TopLeft, Action.Left, Action.BottomLeft, Action.Bottom, Action.BottomRight]
    }
    
    ## Hyperparameters
    SPRINT_RANGE = 0.6
    SHOOT_RANGE_X = 0.7
    SHOOT_RANGE_Y = 0.2
    LONG_RANGE_SHOT_X = 0.4
    LONG_RANGE_SHOT_Y = 0.2
    XG_THRESHOLD = 0.1
    
    ## Games State
    player_pos = obs['left_team'][obs['active']]
    prev_owner = state["prev_owner"]
    ball_owned_team = obs["ball_owned_team"]
    state["prev_owner"] = ball_owned_team
    ball_pos = obs["ball"]
    ball_direction = obs["ball_direction"]
    ball_owned_player = obs["ball_owned_player"]
    goal_post_top = [1, 0.044]
    goal_post_bot = [1, -0.044]
    goal = [1, 0]
    
    ## Game Agent
    if  abs(player_pos[0]) < SPRINT_RANGE and Action.Sprint not in obs['sticky_actions']:
        # Sprint when in the final half of either side of the pitch 
        return Action.Sprint
    elif player_pos[0] > SPRINT_RANGE and Action.Sprint in obs['sticky_actions']:
        # Else run normally
        return Action.ReleaseSprint
    
    ## Game Modes
    if obs["game_mode"] == GameMode.KickOff:
        # KickOff - Make a short pass to the closest teammate
        return direction_check(Action.ShortPass,
                               coordinate_to_direction(ball_pos[:2],
                                                       obs["left_team"][closest_player(player_pos)]))

    elif obs["game_mode"] == GameMode.GoalKick:
        # GoalKick - Make a short pass to the closest teammate or make a long shot 
        teammate = closest_player(player_pos)
        teammate_pos = obs["left_team"][teammate]
        if opponent_on_path(player_pos, teammate_pos):
            # Choose a random place to longpass it
            return direction_check(Action.LongPass,
                                   np.random.choice([Action.TopRight,
                                                     Action.Right,
                                                     Action.BottomRight]))
        else:
            # ShortPass to a close teammate
            return direction_check(Action.ShortPass, 
                                   coordinate_to_direction(ball_pos[:2],
                                                           teammate_pos))

    elif obs["game_mode"] == GameMode.FreeKick:
        # FreeKick - Shoot when close to goal and pass ball otherwise
        # Shoot when close to goal
        if player_pos[0] > SHOOT_RANGE_X and abs(player_pos[1]) < SHOOT_RANGE_Y :
            return direction_check(Action.Shot,
                                   coordinate_to_direction(ball_pos[:2], goal))
        else:
            # When far from goal
            teammate = closest_player(player_pos)
            teammate_pos = obs["left_team"][teammate]            
            if opponent_on_path(player_pos, teammate_pos):
                # LongPass to the Right if the closest player is obstructed by opponent
                return direction_check(Action.LongPass, 
                                       np.random.choice([Action.TopRight,
                                                         Action.Right,
                                                         Action.BottomRight]))
            else:
                # ShortPass to a close teammate
                return direction_check(Action.ShortPass,
                                       coordinate_to_direction(ball_pos[:2],
                                                               teammate_pos))

    elif obs["game_mode"] == GameMode.Corner:
        # Corner - Pass the ball to random points in the box
        directions = [coordinate_to_direction(ball_pos[:2], goal),
                      coordinate_to_direction(ball_pos[:2], [.8, 0])]
        for direction in directions:
            if direction in obs['sticky_actions']:
                return Action.HighPass
        return np.random.choice(directions)

    elif obs["game_mode"] == GameMode.Penalty:
        # Penalty - Randomly choose a direction (either a goal post or the center of the goal)
        directions = [coordinate_to_direction(ball_pos[:2], goal),
                      coordinate_to_direction(ball_pos[:2], goal_post_top),
                      coordinate_to_direction(ball_pos[:2], goal_post_bot)]
        for direction in directions:
            if direction in obs['sticky_actions']:
                return Action.Shot
        return np.random.choice(directions)

    # Offense - When we have the ball
    if obs['ball_owned_player'] == obs['active'] and obs['ball_owned_team'] == 0:        
        # Only for ThrowIn Game Mode does the player have possession
        if obs["game_mode"] == GameMode.ThrowIn:
            # ThrowIn - Make a short pass to the closest teammate
            return direction_check(Action.ShortPass,
                                   coordinate_to_direction(player_pos,
                                                           obs["left_team"][closest_player(player_pos)]))
        
        else :
            # Normal
            # When winning, waste time
            if obs["score"][0] > obs["score"][1] and player_pos[0] < 0:
                teammate = closest_player(player_pos)
                teammate_pos = obs["left_team"][teammate]            
                if opponent_on_path(player_pos, teammate_pos):
                    # LongPass to the Right if the closest player is obstructed by opponent
                    return direction_check(Action.LongPass, 
                                           np.random.choice([Action.TopRight,
                                                             Action.Right,
                                                             Action.BottomRight]))
                else:
                    # ShortPass to a close teammate
                    return direction_check(Action.ShortPass,
                                           coordinate_to_direction(ball_pos[:2],
                                                                   teammate_pos))         
            
            # Calculate goalie position
            goalie_pos = np.array(obs["right_team"][0])
            dist_to_goalie = np.linalg.norm(np.array(player_pos) - goalie_pos)
            dist_goalie_off_line = np.linalg.norm(np.array(goal) - goalie_pos)
            
            # Shoot when close to goal
            if player_pos[0] > SHOOT_RANGE_X and \
                abs(player_pos[1]) < SHOOT_RANGE_Y and \
                player_pos[0] < obs['ball'][0]:
                return direction_check(Action.Shot,
                                       coordinate_to_direction(player_pos, goal))
            
            # Clear the ball when close to our goal
            if player_pos[0] < -SHOOT_RANGE_X and \
                abs(player_pos[1]) < SHOOT_RANGE_Y:
                return direction_check(Action.Shot,
                                       coordinate_to_direction(player_pos, goal))
            
            # Shoot when goali leaves line and close-ish to goal
            if dist_goalie_off_line > constants["goalie_out_of_box_dist"] and \
                player_pos[0] > LONG_RANGE_SHOT_X and \
                abs(player_pos[1]) < LONG_RANGE_SHOT_Y:
                return direction_check(Action.Shot,
                                       coordinate_to_direction(player_pos, goal))
            
            # if close to goal and too wide for shot pass the ball
            if player_pos[0] > SHOOT_RANGE_X and abs(player_pos[1]) > SHOOT_RANGE_Y:
                return direction_check(Action.ShortPass,
                                       coordinate_to_direction(player_pos,
                                                               obs["left_team"][closest_player(player_pos)]))            
            # Shoot when expected goal is quite high
#             if expected_goal(player_pos) > XG_THRESHOLD:
#                 return direction_check(Action.Shot,
#                                        coordinate_to_direction(player_pos, goal))
            
            # Run towards the Goal
            if player_pos[0] > 0:
                return coordinate_to_direction(player_pos, goal)
            
            # Run towards the Right and along the sidelines
            if player_pos[0] < - 0.15:
                if player_pos[1] > 0:
                    return coordinate_to_direction(player_pos, [0, 0.3])
                else:
                    return coordinate_to_direction(player_pos, [0, -0.3])
            
            return Action.Right

    # Defense
    else:
        # When the opponent has the ball
        opponent_pos = np.array(obs["right_team"][ball_owned_player])
        opponent_dir = np.array(obs["right_team_direction"][ball_owned_player])
        dist_to_ball = np.linalg.norm(opponent_pos - np.array(ball_pos[:2]))
        # If the player is close enough to change direction of the ball we should go to the new ball position
        if constants["max_distance_to_influence_ball_sprinting"] > dist_to_ball and np.linalg.norm(opponent_dir) != 0:
            return coordinate_to_direction(player_pos,
                                           predict_ball_pos(ball_pos[:2],
                                                            ball_direction[:2]))

        dist_to_ball = np.linalg.norm(player_pos - np.array(ball_pos[:2]))
        # If the player is close enough to apply pressure, then we should go to the new ball position
        if constants["pressure_threshold"] > dist_to_ball:
            return coordinate_to_direction(player_pos,
                                           predict_ball_pos(ball_pos[:2],
                                                            ball_direction[:2]))
        
        # Run in the way of the player
        if player_pos[0] < opponent_pos[0]:
            return coordinate_to_direction(player_pos, projection(ball_pos, player_pos))

        # Run towards the ball
        return coordinate_to_direction(player_pos, ball_pos[:2])

    # Not sure this is ever reached
    state["prev_player_pos"] = player_pos
    state["prev_ball_pos"] = ball_pos
    
    # Just follow the ball
    return coordinate_to_direction(player_pos, ball_pos[:2])

# Environment Setup

Now we can initialize the environment with the desired scenario (ours or the pre-built ones) and use our agent on it.

In [ ]:
from kaggle_environments import make

scenarios = {0: "11_vs_11_competition",
             1: "11_vs_11_easy_stochastic",
             2: "11_vs_11_hard_stochastic",
             3: "11_vs_11_kaggle",
             4: "11_vs_11_stochastic",
             5: "1_vs_1_easy",
             6: "5_vs_5",
             7: "academy_3_vs_1_with_keeper",
             8: "academy_corner",
             9: "academy_counterattack_easy",
             10: "academy_counterattack_hard",
             11: "academy_empty_goal",
             12: "academy_empty_goal_close",
             13: "academy_pass_and_shoot_with_keeper",
             14: "academy_run_pass_and_shoot_with_keeper",
             15: "academy_run_to_score",
             16: "academy_run_to_score_with_keeper",
             17: "academy_single_goal_versus_lazy",
             18: "custom_scenario_11v11"
             }

# Choose here the scenario of interest
env = make("football", debug=True,
           configuration={"save_video": True, 
                          "scenario_name": scenarios[0], 
                          "running_in_notebook": True})

# Choose here the agents that play
output = env.run(["/kaggle/working/submission.py", "/kaggle/working/evaluate.py"])
scores = output[-1][0]["observation"]["players_raw"][0]["score"]
print("Scores  {0} : {1}".format(*scores))

print('Left player: reward = %s, status = %s, info = %s' % (output[-1][0]['reward'], output[-1][0]['status'], output[-1][0]['info']))
print('Right player: reward = %s, status = %s, info = %s' % (output[-1][1]['reward'], output[-1][1]['status'], output[-1][1]['info']))

# Optional default Game Rendering
env.render(mode="human", width=600, height=400)

# Fancy Animation
[Source](https://www.kaggle.com/jaronmichal/human-readable-visualization)

In [ ]:
import matplotlib.patches as patches
from  matplotlib.patches import Arc
from matplotlib import pyplot as plt
from matplotlib import animation
import matplotlib.patches as mpatches

from enum import Enum

import math

import numpy as np
from IPython.display import HTML

# Change size of figure
plt.rcParams['figure.figsize'] = [20, 16]
# Function that draw the green pitch with white lines
def drawPitch(width, height, color="w"):    
    fig = plt.figure()
    ax = plt.axes(xlim=(-10, width + 10), ylim=(-15, height + 5))
    plt.axis('off')

    # Grass around pitch
    rect = patches.Rectangle((-5,-5), width + 10, height + 10, linewidth=1, edgecolor='gray',facecolor='#3f995b', capstyle='round')
    ax.add_patch(rect)

    # Pitch boundaries
    rect = plt.Rectangle((0, 0), width, height, ec=color, fc="None", lw=2)
    ax.add_patch(rect)

    # Middle line
    plt.plot([width/2, width/2], [0, height], color=color, linewidth=2)

    # Dots
    dots_x = [11, width/2, width-11]
    for x in dots_x:
        plt.plot(x, height/2, 'o', color=color, linewidth=2)

    # Penalty box  
    penalty_box_dim = [16.5, 40.3]
    penalty_box_pos_y = (height - penalty_box_dim[1]) / 2

    rect = plt.Rectangle((0, penalty_box_pos_y), penalty_box_dim[0], penalty_box_dim[1], ec=color, fc="None", lw=2)
    ax.add_patch(rect)
    rect = plt.Rectangle((width, penalty_box_pos_y), -penalty_box_dim[0], penalty_box_dim[1], ec=color, fc="None", lw=2)
    ax.add_patch(rect)

    #Goal box
    goal_box_dim = [5.5, penalty_box_dim[1] - 11 * 2]
    goal_box_pos_y = (penalty_box_pos_y + 11)

    rect = plt.Rectangle((0, goal_box_pos_y), goal_box_dim[0], goal_box_dim[1], ec=color, fc="None", lw=2)
    ax.add_patch(rect)
    rect = plt.Rectangle((width, goal_box_pos_y), -goal_box_dim[0], goal_box_dim[1], ec=color, fc="None", lw=2)
    ax.add_patch(rect)

    #Goals
    rect = plt.Rectangle((0, penalty_box_pos_y + 16.5), -3, 7.5, ec=color, fc=color, lw=2, alpha=0.3)
    ax.add_patch(rect)
    rect = plt.Rectangle((width, penalty_box_pos_y + 16.5), 3, 7.5, ec=color, fc=color, lw=2, alpha=0.3)
    ax.add_patch(rect)

    # Middle circle
    mid_circle = plt.Circle([width/2, height/2], 9.15, color=color, fc="None", lw=2)
    ax.add_artist(mid_circle)


    # Penalty box arcs
    left  = patches.Arc([11, height/2], 2*9.15, 2*9.15, color=color, fc="None", lw=2, angle=0, theta1=308, theta2=52)
    ax.add_patch(left)
    right = patches.Arc([width - 11, height/2], 2*9.15, 2*9.15, color=color, fc="None", lw=2, angle=180, theta1=308, theta2=52)
    ax.add_patch(right)

    # Arcs on corners
    corners = [[0, 0], [width, 0], [width, height], [0, height]]
    angle = 0
    for x,y in corners:
        c = patches.Arc([x, y], 2, 2, color=color, fc="None", lw=2, angle=angle,theta1=0, theta2=90)
        ax.add_patch(c)
        angle += 90
    return fig, ax

WIDTH = 105
HEIGHT = 68
X_RESIZE = WIDTH
Y_RESIZE = HEIGHT / 0.42

class GameMode(Enum):
    Normal = 0
    KickOff = 1
    GoalKick = 2
    FreeKick = 3
    Corner = 4
    ThrowIn = 5
    Penalty = 6

def scale_x(x):
    return (x + 1) * (X_RESIZE/2)

def scale_y(y):
    return (y + 0.42) * (Y_RESIZE/2)

def extract_data(frame):
    # Function that gets game data at each time frame
    res = {}
    obs = frame[0]['observation']['players_raw'][0]
    res["left_team"] = [(scale_x(x), scale_y(y)) for x, y in obs["left_team"]]
    res["right_team"] = [(scale_x(x), scale_y(y)) for x, y in obs["right_team"]]

    ball_x, ball_y, ball_z = obs["ball"]
    res["ball"] = [scale_x(ball_x),  scale_y(ball_y), ball_z]
    res["score"] = obs["score"]
    res["steps_left"] = obs["steps_left"]
    res["ball_owned_team"] = obs["ball_owned_team"]
    res["ball_owned_player"] = obs["ball_owned_player"]
    res["right_team_roles"] = obs["right_team_roles"]
    res["left_team_roles"] = obs["left_team_roles"]
    res["left_team_direction"] = obs["left_team_direction"]
    res["right_team_direction"] = obs["right_team_direction"]
    res["game_mode"] = GameMode(obs["game_mode"]).name
    return res

def draw_team(obs, team, side):
    X = []
    Y = []
    for x, y in obs[side]:
        X.append(x)
        Y.append(y)
    team.set_data(X, Y)

def draw_ball(obs, ball):
    # Function that draws the ball on the field    
    ball.set_markersize(10 + 5 * obs["ball"][2]) # Scale size of ball based on height
    ball.set_data(obs["ball"][:2])

def draw_ball_owner(obs, ball_owner, team_active):
    # Function that highlights the player in possession of the ball    
    if obs["ball_owned_team"] == 0:
        x, y = obs["left_team"][obs["ball_owned_player"]]
        ball_owner.set_data(x, y)
        team_active.set_data(WIDTH / 4 + 7, -7)
        team_active.set_markerfacecolor("red")
    elif obs["ball_owned_team"] == 1:
        x, y = obs["right_team"][obs["ball_owned_player"]]
        ball_owner.set_data(x, y)
        team_active.set_data(WIDTH / 4 + 50, -7)
        team_active.set_markerfacecolor("blue")
    else:
        ball_owner.set_data([], [])
        team_active.set_data([], [])
    
def draw_players_directions(obs, directions, side):
    # Function that direction in which players run
    index = 0
    if "right" in side:
        index = 11
    for i, player_dir in enumerate(obs[f"{side}_direction"]):
        x_dir, y_dir = player_dir
        dist = math.sqrt(x_dir ** 2 + y_dir ** 2) + 0.00001 # to prevent division by 0
        x = obs[side][i][0]
        y = obs[side][i][1] 
        directions[i + index].set_data([x, x + x_dir / dist ], [y, y + y_dir / dist])

# Settup for the animation
fig, ax = drawPitch(WIDTH, HEIGHT)
ax.invert_yaxis()

ball_owner, = ax.plot([], [], 'o', markersize=30,  markerfacecolor="yellow", alpha=0.5)
team_active, = ax.plot([], [], 'o', markersize=30,  markerfacecolor="blue", markeredgecolor="None")

team_left, = ax.plot([], [], 'o', markersize=20, markerfacecolor="r", markeredgewidth=2, markeredgecolor="white")
team_right, = ax.plot([], [], 'o', markersize=20,  markerfacecolor="b", markeredgewidth=2, markeredgecolor="white")

ball, = ax.plot([], [], 'o', markersize=10,  markerfacecolor="black", markeredgewidth=2, markeredgecolor="white")
text_frame = ax.text(-5, -5, '', fontsize=25)
match_info = ax.text(105 / 4 + 10, -5, '', fontsize=25)
game_mode = ax.text(105 - 25, -5, '', fontsize=25)
goal_notification = ax.text(105 / 4 + 10, 0, '', fontsize=25)

# Drawing of directions definitely can be done in a better way
directions = []
for i in range(22):
    direction, = ax.plot([], [], color='yellow', lw=3)
    directions.append(direction)

drawings = [team_active, ball_owner, team_left, team_right, ball, text_frame, match_info, game_mode, goal_notification]

def init():
    team_left.set_data([], [])
    team_right.set_data([], [])
    ball_owner.set_data([], [])
    team_active.set_data([], [])
    ball.set_data([], [])
    return drawings 

def animate(i):
    global prev_score_a, prev_score_b
    obs = extract_data(output[i])

    # Draw info about ball possesion
    draw_ball_owner(obs, ball_owner, team_active)

    # Draw players
    draw_team(obs, team_left, "left_team")
    draw_team(obs, team_right, "right_team")

    draw_players_directions(obs, directions, "left_team")
    draw_players_directions(obs, directions, "right_team")

    draw_ball(obs, ball)

    # Draw textual informations
    text_frame.set_text(f"Frame: {i}/{obs['steps_left'] + i - 1}")
    game_mode.set_text(f"Game mode: {obs['game_mode']}")

    score_a, score_b = obs["score"]
    match_info.set_text(f"Left team {score_a} : {score_b} Right Team")

    return drawings  

In [ ]:
# Build the animation and draw it
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=game_duration, interval=100, blit=True)

HTML(anim.to_html5_video())